**Clean and buffer MVUM road data**

In [3]:
import geopandas as gpd
import sys
from pathlib import Path

notebook_path = Path.cwd()
project_root = notebook_path.parent
sys.path.append(str(project_root))


from scripts import config

layer_name = 'MVUM_Roads_CO'

In [15]:
gdf = gpd.read_file(config.GDB_PATH, layer=layer_name)

print(gdf.loc[0])

FID_Motor_Vehicle_Use_Map_3A_Roads__Feature_Layer_                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

**Cleaning**

First, let's drop unnecessary columns.

In [ ]:
gdf = gdf[['geometry', 'ID', 'NAME', 'juris_valid', 'status_valid']]